# Import

In [1]:
import numpy as np
import os
import time
import sklearn
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import layers
from tensorflow.python.keras.engine import keras_tensor
from tensorflow.keras.layers import experimental
from tensorflow.keras.layers.experimental import preprocessing
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Layer, Input, Conv1D, Conv2D, Conv3D, Reshape, BatchNormalization, Activation, Dropout, Reshape
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, roc_auc_score, roc_curve
import pandas as pd
import tensorflow.keras.backend as K
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, roc_auc_score, roc_curve
#import statsmodels.api as sm
import statsmodels.formula.api as smf
from math import sqrt
from tqdm.notebook import tqdm
import seaborn as sns
import tensorflow.keras.layers as KL
import tensorflow.keras.models as KM
import datetime, json 
#import xgboost
#import shap
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from IPython.display import SVG
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Dense, Flatten,MaxPooling2D
import collections
import random
import os
import time
import json
import itertools
from PIL import Image

2022-11-16 13:37:55.279016: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/home/dsl001/anaconda3/envs/yeop/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.7.0 and strictly below 2.10.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [3]:
#from shapreg import removal, stochastic_games, shapley
#from shapreg.utils import crossentropyloss
from numpy import empty, uint8
from math import factorial
#import sage
from sklearn.model_selection import train_test_split
import tensorflow as tf
from numpy.random import seed
seed(1)
from tensorflow.python.client import device_lib
from tensorflow.keras.utils import get_custom_objects
from tensorflow.python.client import device_lib

In [4]:
from tensorflow.keras.layers import experimental
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import Normalization, Resizing

In [5]:
tf.__version__

'2.4.1'

# 함수정의

## -Basic

In [6]:
def division(data):
    '''
    train:validation:test=6:2:2 데이터 분할
    '''
    train_size = int(len(data)*0.6)
    val_size = int(len(data)*0.8)
    
    data_train = data[0:train_size]
    data_val = data[train_size:val_size]
    data_test = data[val_size:len(data)]
    
    return data_train, data_val, data_test


def look_back(X, Y, a):
    '''
    시계열 분석을 위해 timestep과 lag를 적용하여 데이터셋을 생성
    '''
    X_lb = np.zeros((len(X) - a, a, 40,40,11))
    for l in range(len(X) - a): 
        for r in range(a):
            X_lb[l, r] = X[l + r, 0]
    Y = Y[a:]
    
    return X_lb, Y


def get_dropout(input_tensor, p=0.5, mc=False):
    '''
    몬테카를로 드롭아웃
    기존 학습 과정에서만 적용되었던 드롭아웃을 테스트 과정에서도 적용되도록 한다.
    '''
    if mc:
        return Dropout(p)(input_tensor, training=True)
    else:
        return Dropout(p)(input_tensor)

def look_back1(X, Y, a):
    '''
    시계열 분석을 위해 timestep과 lag를 적용하여 데이터셋을 생성
    '''
    X_lb = np.zeros((len(X) - a, a, 40, 28, 11))
    for l in range(len(X) - a): 
        for r in range(a):
            X_lb[l, r] = X[l + r, 0]
    Y = Y[a:]
    
    return X_lb, Y

# 커스텀 객체 선언
# 참고 : https://hwiyong.tistory.com/375
def mish(x):
    return x * K.tanh(K.softplus(x))

get_custom_objects().update({'mish': mish})


## - ViT architecture

In [7]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            "num_patches " : self.num_patches,
            "projection " : self.projection ,
            "position_embedding" : self.position_embedding
        })
        return config

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded
    
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            "patch_size" : self.patch_size
        })
        return config
        
    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches
    
def multilayer_perceptron(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

#  VIT reshape ver

In [8]:
def vit_r(tm):
    attention_out =[]
    encoded_patch = []
    output = []
    attention_weights = []
    with tf.device('/device:GPU:0'):
        input_tensor = Input(shape=(tm,40, 40, 11)) ## 행,열,
        x = Conv3D(11,(tm,1,1), padding='valid', kernel_regularizer=l2(0.01))(input_tensor)
        x = Reshape((40, 40,11))(x)
        x_1 = Conv2D(1,(1,1), padding='valid', kernel_regularizer=l2(0.01))(x)
        augmented = data_augmentation(x_1)
        patches = Patches(patch_size)(augmented)
        encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)
    
        for _ in range(transformer_layers):

            x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
            
            #a, b = scaled_dot_product_attention(x1, x1, x1, 1)
            #output.append(a)
            #attention_weights.append(b)
            
            attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim, dropout=0.1)(x1, x1)
            attention_out.append(attention_output)
            
            x2 = layers.Add()([attention_output, encoded_patches])
            

            x3 = layers.LayerNormalization(epsilon=1e-6)(x2)

            x3 = multilayer_perceptron(x3, hidden_units=transformer_units, dropout_rate=0.1)
            
            encoded_patches = layers.Add()([x3, x2])
            encoded_patch.append(encoded_patches)
            #encoded_patches = encoded_patches.layer.Add(keras.layers.Embedding(input_shape=(None,),input_dim=10,output_dim=5,mask_zero=True,))

            #TrigPosEmbedding(output_dim=5,mode=TrigPosEmbedding.MODE_ADD,)
        
       # x = Reshape((40, 40,-1))(encoded_patches)

        x = Conv2D(64,(1,1), padding='valid', kernel_regularizer=l2(0.01))(x)
        x = BatchNormalization()(x)
        x = Activation(mish)(x)
        x = get_dropout(x, p=0.25, mc=True)
        
        x = Conv2D(32, (1,1), padding='same', kernel_regularizer=l2(0.01))(x)
        x = BatchNormalization()(x)
        x = Activation(mish)(x)
        x = get_dropout(x, p=0.25, mc=True)
    
        x = Conv2D(16, (1,1), padding='same', kernel_regularizer=l2(0.01))(x)
        x = BatchNormalization()(x)
        x = Activation(mish)(x)
        x = get_dropout(x, p=0.25, mc=True)
    
        x = Conv2D(8, (1,1), padding='same', kernel_regularizer=l2(0.01))(x)
        x = BatchNormalization()(x)
        x = Activation(mish)(x)
        x = get_dropout(x, p=0.25, mc=True)
        x = Conv2D(1, (1, 1), padding='same')(x)
        output_tensor = keras.layers.Reshape((1,-1))(x)
        #x = Flatten()(x)
        
    
        #output_tensor = Conv1D(1,(1,1),padding='same')(x)
        #output_tensor = (x)
        model = Model(input_tensor, output_tensor)

        #model.summary()
        #early_stopping = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 50, mode = 'min')
        #mc = ModelCheckpoint('./Result', monitor='val_loss', mode='min', save_best_only=True)
        #model.compile(loss='mean_squared_error', optimizer= 'adam')
        
        #history = model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 20, batch_size=64, callbacks=[early_stopping, mc])
    return model

## Framework

In [9]:
def vit(tm):
    attention_out =[]
    encoded_patch = []
    output = []
    attention_weights = []
    with tf.device('/device:GPU:0'):
        input_tensor = Input(shape=(tm,40, 40, 11)) ## 행,열,
        x = Conv3D(11,(tm,1,1), padding='valid', kernel_regularizer=l2(0.01))(input_tensor)
        x = Reshape((40, 40,11))(x)
        x_1 = Conv2D(1,(1,1), padding='valid', kernel_regularizer=l2(0.01))(x)
        augmented = data_augmentation(x_1)
        patches = Patches(patch_size)(augmented)
        encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)
    
        for _ in range(transformer_layers):

            x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
            
            #a, b = scaled_dot_product_attention(x1, x1, x1, 1)
            #output.append(a)
            #attention_weights.append(b)
            
            attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim, dropout=0.1)(x1, x1)
            attention_out.append(attention_output)
            
            x2 = layers.Add()([attention_output, encoded_patches])
            

            x3 = layers.LayerNormalization(epsilon=1e-6)(x2)

            x3 = multilayer_perceptron(x3, hidden_units=transformer_units, dropout_rate=0.1)
            
            encoded_patches = layers.Add()([x3, x2])
            encoded_patch.append(encoded_patches)
            #encoded_patches = encoded_patches.layer.Add(keras.layers.Embedding(input_shape=(None,),input_dim=10,output_dim=5,mask_zero=True,))

            #TrigPosEmbedding(output_dim=5,mode=TrigPosEmbedding.MODE_ADD,)
        
       # x = Reshape((40, 40,-1))(encoded_patches)

        x = Conv2D(64,(1,1), padding='valid', kernel_regularizer=l2(0.01))(x)
        x = Activation(mish)(x)
        x = get_dropout(x, p=0.25, mc=True)
        
        x = Conv2D(32, (1,1), padding='same', kernel_regularizer=l2(0.01))(x)
        x = Activation(mish)(x)
        x = get_dropout(x, p=0.25, mc=True)
    
        x = Conv2D(16, (1,1), padding='same', kernel_regularizer=l2(0.01))(x)
        x = Activation(mish)(x)
        x = get_dropout(x, p=0.25, mc=True)
    
        x = Conv2D(8, (1,1), padding='same', kernel_regularizer=l2(0.01))(x)
        x = Activation(mish)(x)
        x = get_dropout(x, p=0.25, mc=True)
        
        x = Conv2D(1, (1, 1), padding='same')(x)
        x = Flatten()(x)
        output_tensor = Dense(1)(x)
        model = Model(input_tensor, output_tensor)
        #model.summary()
        #early_stopping = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 50, mode = 'min')
        #mc = ModelCheckpoint('./Result', monitor='val_loss', mode='min', save_best_only=True)
        #model.compile(loss='mean_squared_error', optimizer= 'adam')
        
        #history = model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 20, batch_size=64, callbacks=[early_stopping, mc])
    return model

# Data Preprocessing

In [10]:
'''
디렉토리에서 데이터를 불러온후 Reshape를 통해 이미지 형식으로 변환
'''

path_dir = "./IDWDATA_40_28size/"
file_list = ["CO_sizereset.csv",'NO2_sizereset.csv', 'O3_sizereset.csv', 'PM10_sizereset.csv','PM25_sizereset.csv', 'SO2_sizereset.csv', 'temp_sizereset.csv', 'wind_x_sizereset.csv','wind_y_sizereset.csv','windspeed_sizereset.csv','rainfall_sizereset.csv']

concat = []
dataset = []
flg = 0
i = 0
for f in tqdm(file_list):
    df = pd.read_csv(path_dir + f, engine='python')
    del df['Unnamed: 0']

    df_flat = df.values.flatten()
    df_flat_rs = df_flat.reshape(len(df_flat), 1)

    if (i == 0):
        concat = df_flat_rs
    else:
        concat = np.concatenate((concat, df_flat_rs), axis=1)
    i = i+1

if(flg==0):
    dataset = concat
else:
    dataset = np.concatenate((dataset, concat), axis=0)
flg = flg+1

# reshape here
#data = np.reshape(dataset, ((8760*2),1,40, 28, 11))

  0%|          | 0/11 [00:00<?, ?it/s]

## data 

In [11]:
data = np.reshape(dataset, ((8760*2),1, 40, 28, 11))

# Zero padding

In [12]:
zero_data=np.zeros(((8760*2),1,40,40,11))
zero_data[:,:,:,6:34,:] = data
zero_data.shape
np.shape(zero_data)

(17520, 1, 40, 40, 11)

In [13]:
x_data=zero_data[:,:,:,:,:]
y_data = zero_data[:,:,:,:,3]
np.shape(y_data)

(17520, 1, 40, 40)

In [14]:
# y_data = y_data.reshape((-1),1,40,40,1)
# y_data = y_data[:100,:,:,:,:]
# np.shape(y_data)

In [15]:
x_data, y_data = look_back(x_data, y_data,12) 

In [16]:
np.shape(x_data)

(17508, 12, 40, 40, 11)

In [17]:
x_train, x_val, x_test = division(x_data)
y_train, y_val, y_test = division(y_data)

print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_val.shape}")

x_train shape: (10504, 12, 40, 40, 11) - y_train shape: (10504, 1, 40, 40)
x_test shape: (3502, 12, 40, 40, 11) - y_test shape: (3502, 1, 40, 40)


# ViT parameter 

In [18]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 64
num_epochs = 1000
image_size = 40  # 입력 이미지의 크기를 이 크기로 조정합니다.
patch_size = 1    # 입력 이미지에서 추출할 패치의 크기
num_patches = (image_size // patch_size) ** 2
projection_dim = 1
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # 변압기 층의 크기
transformer_layers = 8
mlp_head_units = [2048, 1024]  # 최종 분류기의 조밀 레이어 크기

## Norm

In [19]:
data_augmentation = keras.Sequential(
    [
        experimental.preprocessing.Normalization(),
        experimental.preprocessing.Resizing(image_size, image_size)
    ],
    name="data_augmentation",
)

2022-11-16 13:41:15.826772: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-16 13:41:15.829104: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-11-16 13:41:16.028464: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:18:00.0 name: TITAN RTX computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.62GiB deviceMemoryBandwidth: 625.94GiB/s
2022-11-16 13:41:16.031657: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:3b:00.0 name: TITAN RTX computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.62GiB deviceMemoryBandwidth: 625.94GiB/s
2022-11-16 13:41:16.034776: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:86:00.0 name: TITAN RTX computeCapabilit

In [20]:
#data_augmentation.layers[0].adapt(x_train)

In [21]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 텐서플로가 세 번째 GPU만 사용하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[2], 'GPU')
    except RuntimeError as e:
        print(e)

Visible devices cannot be modified after being initialized


## Data load for backup

In [168]:
x_data = zero_data
y_data = zero_data[:,:,:,:,3] # 3 = PM10, 4 = PM2.5
y_data = y_data.reshape((-1),1,40,40,1)
x_data, y_data = look_back(x_data, y_data, 1)
x_train, x_val, x_test = division(x_data)
y_train, y_val, y_test = division(y_data)

# dsw=[]
# for i in range(len(x_train)):
#     dsw.append(x_train[i][0][20][20])
# x_train=np.array(dsw)

# dsw=[]
# for i in range(len(x_val)):
#     dsw.append(x_val[i][0][20][20])
# x_val=np.array(dsw)

# dsw=[]
# for i in range(len(x_test)):
#     dsw.append(x_test[i][0][20][20])
# x_test=np.array(dsw)


dsw=[]
for i in range(len(y_train)):
    dsw.append(y_train[i][0][20][20])
y_train=np.array(dsw)

dsw=[]
for i in range(len(y_val)):
    dsw.append(y_val[i][0][20][20])
y_val=np.array(dsw)

dsw=[]
for i in range(len(y_test)):
    dsw.append(y_test[i][0][20][20])
y_test=np.array(dsw)

# y_train = y_train.reshape(y_train.shape[0]).astype('float32')
# y_val = y_val.reshape(y_val.shape[0]).astype('float32')
# y_test = y_test.reshape(y_test.shape[0]).astype('float32')

In [23]:
y_val.shape

(3504, 1)

# Fit

In [ ]:
with tf.device('/device:GPU:1'):
    for tm in tqdm([1]):
        for lag in tqdm([1]):
            early_stopping = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 50, mode = 'min')
        #mc = ModelCheckpoint('./Result', monitor='val_loss', mode='min', save_best_only=True)     
            mc = ModelCheckpoint('./Result/tasd.h5'.format(str(tm), str(lag)), monitor='val_loss', mode='min', save_best_only=True)
            model = vit(tm)
            model.compile(loss='mean_squared_error', optimizer='adam')
            model.fit(x_train, y_train, epochs=2000, batch_size=64, validation_data=(x_val, y_val), callbacks=[early_stopping, mc])

# ICNN - ViT model load 

In [26]:
#from keras.models import load_model
model = load_model('./Result/tasd.h5')

# Shaply value

In [27]:
import shap

In [28]:
x_test.shape

(3504, 1, 40, 40, 11)

In [38]:
with tf.device("/device:GPU:1"):
    explainer = shap.GradientExplainer(model, x_train)

In [45]:
with tf.device('/device:GPU:3'):
    shap_values = explainer.shap_values(x_test[0:3])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [39]:
np.shape(shap_values)

(1, 3, 1, 40, 40, 11)

In [40]:
w=np.reshape(shap_values,(-1,11))

In [41]:
np.shape(w)

(4800, 11)

In [134]:
s=shap_values[0][0][0][19][19]

In [136]:
for i in s:
    print(i)

In [141]:
p=[1.32988362e-05, -2.87315856e-07,  2.72235661e-06, -1.37084152e-03,
        1.04025406e-04,  1.16835359e-08, -3.36589845e-04, -9.09998654e-09,
       -2.01348871e-09,  1.49796382e-04,  2.24860687e-05]
for i in range(len(p)):
    p[i] = p[i] * -1
print(p)

[-1.32988362e-05, 2.87315856e-07, -2.72235661e-06, 0.00137084152, -0.000104025406, -1.16835359e-08, 0.000336589845, 9.09998654e-09, 2.01348871e-09, -0.000149796382, -2.24860687e-05]


In [142]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

industry =df.columns
fig = plt.figure(figsize=(12, 8),dpi=1000)
ax = fig.add_subplot(111)
plt.xlabel('Shap Value')
plt.ylabel('Variables')
ypos = ["CO",'NO2', 'O3', 'PM10','PM2.5', 'SO2', 'temperature', 'wind_x','wind_y','windspeed','rainfall']
colors = ['#e35f62','dodgerblue','#e35f62','dodgerblue', '#e35f62','#e35f62','dodgerblue','dodgerblue','dodgerblue','#e35f62','#e35f62',]
rects = plt.barh(ypos, p, color=colors, align='edge', edgecolor='#eee', height=0.5)
plt.yticks(ypos, industry)
plt.show()

ValueError: The number of FixedLocator locations (11), usually from a call to set_ticks, does not match the number of ticklabels (28).

# performance calculation

In [147]:
y_predict = model.predict(x_test)

In [138]:
def print_static_rmse(actual, predicted, start_from=0,verbose=0):
    rmse = np.sqrt(mean_squared_error(actual[start_from:],predicted[start_from:]))
    #rmsle = np.sqrt(mean_squared_log_error(actual[start_from:],predicted[start_from:]))
    std_dev = actual[start_from:].std()
    if verbose == 1:
        print('    RMSE = %0.2f' %rmse)
        print('    Std Deviation of Actuals = %0.2f' %(std_dev))
        print('    Normalized RMSE = %0.1f%%' %(rmse*100/std_dev))
    return rmse, rmse/std_dev

In [148]:
rmse, rmst = print_static_rmse(y_test, y_predict, 0, 0)
print(rmse,rmst)

3.484278761193373 0.19982435790358177


# attention map

In [149]:
def scaled_dot_product_attention(query, key, value, mask):

    matmul_qk = tf.matmul(query, key, transpose_b=True)

    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    if mask is not None:
        logits += (mask * -1e9)

    attention_weights = tf.nn.softmax(logits, axis=-1)

    output = tf.matmul(attention_weights, value)

    return output, attention_weights

In [170]:
#y_predict = model.predict(x_test)
y_predict = y_predict.reshape(-1, 1).astype('float32')
y_test = y_test.reshape(-1, 1).astype('float32')
x_test = x_test.reshape(-1, 1).astype('float32')

In [177]:
a,b=scaled_dot_product_attention(y_test[:1600], y_predict[:1600], y_test[:1600], 1)

In [178]:
c=list(itertools.chain.from_iterable(a))
c=np.reshape(c,(40,40))
pd.DataFrame(c).to_csv('sample.csv')
ime = Image.fromarray(c)

In [180]:
sns.heatmap(ime, center=60, annot=False, fmt="f", cmap='RdYlGn_r')
plt.axis("off")
plt.show()

In [181]:
sns.heatmap(ime, annot=False, fmt="d", cmap='RdYlGn_r')
plt.axis("off")
#plt.show()